In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

<h3>Connecting to OpenAI</h3>
The next cell loads environment variables from the .env file and connects to OpenAI.I

In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

<h3>Call to a Frontier model</h3>

In [4]:
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


In [5]:
# headers when fetching websites:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a cheapest hotel name in Berlin city for one night stay for couple in march with hight rates, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled called ----- {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a cheapest hotel in Berlin city for one night stay for couple this website in markdown. \
If it includes news or announcements, then remove those.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [12]:

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [13]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://www.booking.com/")

```markdown
### Cheapest Hotel in Berlin for One Night Stay in March

#### Hotel Name: **Ibis Budget Berlin City Potsdamer Platz**

- **Price**: Check the website for the latest rates.
- **Description**: A budget-friendly option perfect for couples looking for a cozy stay in central Berlin.
```
